In [2]:
number = 'six'

import os
import sys
import tflearn
import numpy as np
from tqdm import tqdm

sys.path.append('utils')

from plugin_feature_extractor import PluginFeatureExtractor
import numpy as np
from utility_functions import get_batches, get_stats, display_stats, plot_error, write_wavs
from tqdm import trange

learning_rate = 0.001
training_iters = 13
batch_size = 32
number_hidden = 256
np.random.seed(8)

checkpoint = number + '_dexed_ieee.tfl.ckpt-361000'
data_folder = 'data/overriden/' + number + '_operator/'

overriden_parameters = np.load(data_folder + "overriden_parameters.npy").tolist()
desired_features = [i for i in range(8, 21)]

algorithm_number = 18
alg = (1.0 / 32.0) * float(algorithm_number - 1) + 0.001

overriden_parameters = [(0, 1.0), (1, 0.0), (2, 1.0), (3, 0.0), (4, alg)]
other_params = [((i + 5), 0.5) for i in range(17)]
operator_one = [((i + 23), 0.0) for i in range(22)]
operator_two = [((i + 45), 0.0) for i in range(22)]
operator_thr = [((i + 67), 0.0) for i in range(22)]
operator_fou = [((i + 89), 0.0) for i in range(22)]
operator_fiv = [((i + 111), 0.0) for i in range(22)]
operator_six = [((i + 133), 0.0) for i in range(22)]

overriden_parameters.extend(other_params)
if number is 'one':
    overriden_parameters.extend(operator_two)
    overriden_parameters.extend(operator_thr)
    overriden_parameters.extend(operator_fou)
    overriden_parameters.extend(operator_fiv)
    overriden_parameters.extend(operator_six)
elif number is 'two':
    overriden_parameters.extend(operator_thr)
    overriden_parameters.extend(operator_fou)
    overriden_parameters.extend(operator_fiv)
    overriden_parameters.extend(operator_six)
elif number is 'three':
    overriden_parameters.extend(operator_fou)
    overriden_parameters.extend(operator_fiv)
    overriden_parameters.extend(operator_six)
elif number is 'four':
    overriden_parameters.extend(operator_fiv)
    overriden_parameters.extend(operator_six)
elif number is 'five':
    overriden_parameters.extend(operator_six)
elif number is 'six':
    print "DUH"
    
extractor = PluginFeatureExtractor(midi_note=24, note_length_secs=0.4,
                                   desired_features=[i for i in range(8, 21)],
                                   overriden_parameters=overriden_parameters,
                                   render_length_secs=0.7,
                                   pickle_path="utils/normalisers",
                                   warning_mode="ignore", normalise_audio=False)

path = "/home/tollie/Development/vsts/dexed/Builds/Linux/build/Dexed.so"
extractor.load_plugin(path)

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
DUH
Successfully loaded plugin.


/usr/local/lib/python2.7/dist-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
test_size = 30
train_x = np.load(data_folder + "train_x.npy")
train_y = np.load(data_folder + "train_y.npy")
test_x = np.load(data_folder + "test_x.npy")[0:test_size]
test_y = np.load(data_folder + "test_y.npy")[0:test_size]

features_cols = test_x[0].shape[0]
features_rows = test_x[0].shape[1]
parameter_size = test_y[0].shape[0]

print test_x.shape
print test_y.shape

(30, 27, 13)
(30, 133)


In [4]:
from tflearn.layers.recurrent import bidirectional_rnn, BasicLSTMCell
from tflearn.layers.core import dropout

net = tflearn.input_data([None, features_cols, features_rows])
net = bidirectional_rnn(net, BasicLSTMCell(number_hidden),
                        BasicLSTMCell(number_hidden))
net = dropout(net, 0.8)
net = tflearn.fully_connected(net, 64, activation='elu',
                              regularizer='L2', weight_decay=0.001)
net = tflearn.highway(net, 64, activation='elu',
                      regularizer='L2', weight_decay=0.001,
                      transform_dropout=0.8)
net = tflearn.highway(net, 64, activation='elu',
                      regularizer='L2', weight_decay=0.001,
                      transform_dropout=0.8)
net = tflearn.highway(net, 64, activation='elu',
                      regularizer='L2', weight_decay=0.001,
                      transform_dropout=0.8)
net = tflearn.highway(net, 64, activation='elu',
                      regularizer='L2', weight_decay=0.001,
                      transform_dropout=0.8)
net = tflearn.highway(net, 64, activation='elu',
                      regularizer='L2', weight_decay=0.001,
                      transform_dropout=0.8)
net = tflearn.highway(net, 64, activation='elu',
                      regularizer='L2', weight_decay=0.001,
                      transform_dropout=0.8)
net = tflearn.fully_connected(net, parameter_size, activation='elu')
net = tflearn.regression(net, optimizer='adam', learning_rate=learning_rate,
                         loss='mean_square')
model = tflearn.DNN(net, tensorboard_verbose=0,
                    checkpoint_path=(data_folder + number + '_dexed_ieee.tfl.ckpt'))

In [5]:
training = True

if training:
    model.fit(train_x, train_y, validation_set=((test_x, test_y)),
              show_metric=True, batch_size=batch_size, n_epoch=training_iters,
              snapshot_epoch=True, snapshot_step=1000, run_id='granulator_lstm')
else:
    model.load(data_folder + checkpoint)

Training Step: 24374  | total loss: 0.08347 | time: 43.357s
| Adam | epoch: 013 | loss: 0.08347 - acc: 0.0214 -- iter: 59968/60000
Training Step: 24375  | total loss: 0.08362 | time: 44.384s
| Adam | epoch: 013 | loss: 0.08362 - acc: 0.0193 | val_loss: 0.08227 - val_acc: 0.0333 -- iter: 60000/60000
--
INFO:tensorflow:/home/tollie/Development/TensorFlowSynthProgrammers/data/overriden/six_operator/six_dexed_ieee.tfl.ckpt-24375 is not in all_model_checkpoint_paths. Manually adding it.


INFO:tensorflow:/home/tollie/Development/TensorFlowSynthProgrammers/data/overriden/six_operator/six_dexed_ieee.tfl.ckpt-24375 is not in all_model_checkpoint_paths. Manually adding it.


INFO:tensorflow:/home/tollie/Development/TensorFlowSynthProgrammers/data/overriden/six_operator/six_dexed_ieee.tfl.ckpt-24375 is not in all_model_checkpoint_paths. Manually adding it.


INFO:tensorflow:/home/tollie/Development/TensorFlowSynthProgrammers/data/overriden/six_operator/six_dexed_ieee.tfl.ckpt-24375 is not in all_model_checkpoint_paths. Manually adding it.


In [7]:
def get_abs_error(pred, actual):
    """Get the absolute error between the two arrays."""
    return np.add.reduce(np.abs(pred - actual).flatten())

feature_distances = [] 
parameter_distances = [] 
predicted_patches = []
actual_patches = []
for i in tqdm(range(len(test_x)), desc="Testing the model"):
    features_example = test_x[i].reshape((-1, features_cols, features_rows))
    predicted_patch = np.array(model.predict(features_example)).reshape((parameter_size))
    predicted_full_patch = extractor.partial_patch_to_patch(predicted_patch)
    predicted_patch_with_indices = extractor.add_patch_indices(predicted_full_patch)
    predicted_features = extractor.get_features_from_patch(predicted_patch_with_indices)
    actual_features = test_x[i]
    
    feature_distance = get_abs_error(predicted_features, actual_features)
    parameter_distance = get_abs_error(predicted_patch, test_y[i])
    
    feature_distances += [feature_distance]
    parameter_distances += [parameter_distance]
    predicted_patches += [predicted_patch]
    actual_patches += [test_y[i]]

Testing the model:   0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X

In [8]:
def mean_ieee(errors, name):
    print name + ": " + str(round(np.mean(errors), 1)) + " : " + str(round(np.mean(np.mean(sorted(errors)[0:23])), 1))

mean_ieee(feature_distances, "BiDi")

def rounded_list(errors):
    print [round(e, 3) for e in errors]

print ""
print number + ":"
rounded_list(feature_distances)

BiDi: 22.6 : 18.0

six:
[73.14, 26.02, 41.71, 0.641, 24.475, 22.228, 21.605, 27.217, 27.289, 22.332, 26.05, 22.734, 26.983, 19.795, 29.839, 20.378, 2.767, 17.046, 0.939, 31.623, 26.413, 22.316, 24.291, 2.494, 25.083, 0.902, 22.312, 20.42, 16.12, 32.655]
